
<img src="https://minas.medellin.unal.edu.co/eventos/latwaves2018/images/logo_unal.png" alt="unal logo" width="500"/>

#**Trabajo Final de Maestría**
##Ingeniería de Sistemas y Computación - Perfil profundización.
Joan Gabriel Bofill Barrera

---


Inspirado en el trabajo de Jonathan Bown: https://www.kaggle.com/code/jonbown/feedback-prize-candidate-model-exploration/notebook

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from textblob import TextBlob
import spacy
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Normalizer
from tqdm import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from string import punctuation
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
trans = preprocessing.MinMaxScaler()
from collections import Counter
from sklearn.model_selection import train_test_split

In [ ]:
# Importar los datos de drive
from google.colab import drive
from google.colab import files
import io
drive.mount('/content/drive')

train = pd.read_csv('/content/drive/MyDrive/Tesis/train_modificado.csv')
test = pd.read_csv('/content/drive/MyDrive/Tesis/test_modificado.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Normalización**

In [ ]:
#Toma las características extraídas
variables = [var for var in train.columns if var not in ['full_text', 'text_id', 'cohesion','syntax','vocabulary','phraseology','grammar','conventions','Unnamed: 0','diff']]

In [ ]:
#Lista de variables
variables

['word_count',
 'n_capital',
 'n_punct',
 'spelling_errors',
 'n_unique',
 'n_unique_n_stop',
 'n_n_word',
 'noun_phrase_count',
 'PRON',
 'VERB',
 'SCONJ',
 'NOUN',
 'AUX',
 'ADP',
 'PUNCT',
 'PART',
 'CCONJ',
 'ADV',
 'DET',
 'ADJ',
 'SPACE',
 'PROPN',
 'NUM',
 'INTJ',
 'SYM',
 'X',
 'polarity',
 'subjectivity',
 'spell_score',
 'av_sent_len',
 'max_sent_len',
 'min_sent_len',
 'med_sent_len',
 'std_sent_len',
 'num_sent',
 'compound',
 'negative',
 'positive',
 'neutral',
 'char_len']

In [ ]:
#Normaliza las variables para el conjunto de entrenamiento
X_train_custom=trans.fit_transform(train[variables].values)
X_train_custom.shape

(3128, 40)

In [ ]:
#Normaliza las variables para el conjunto de prueba
X_test_custom=trans.fit_transform(test[variables].values)
X_test_custom.shape

(783, 40)

In [ ]:
#Vectorización
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['full_text'])
X_test = vectorizer.transform(test['full_text'])

In [ ]:
X_train=np.asarray(X_train.todense())
X_test=np.asarray(X_test.todense())

In [ ]:
#Targets
y_vars = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

In [ ]:
y_train = train[y_vars]
y_test = test[y_vars]

# **Métrica**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def RMSE(y, y_hat):
    return np.sqrt(mean_squared_error(y,y_hat))

def MCRMSE(y_df, y_hat_df):
    size = y_df.shape[1]
    mcrmse = 0
    for i in range(size):
        mcrmse+=RMSE(y_df.iloc[:, i], y_hat_df[:, i])
    return mcrmse/size

#**Modelos Lineales**

## 1. LGBM

**LGBM Multi Output Regressor para X_train**




In [ ]:
params_lgb = {
    "n_estimators": 1000,
    "verbose": -1
}

In [ ]:
%%time
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train, y_train)

CPU times: user 7min 7s, sys: 1.11 s, total: 7min 8s
Wall time: 1min 48s


MultiOutputRegressor(estimator=LGBMRegressor(n_estimators=1000, verbose=-1))

In [ ]:
y_pred = model.predict(X_test)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.5537921813304042
0.4444616758999591


In [ ]:
y_pred

array([[2.71323644, 2.70287404, 2.77336631, 2.86117229, 2.52525741,
        2.79640703],
       [3.17290698, 2.93772009, 3.00023009, 2.87116396, 2.98541604,
        2.90655578],
       [3.01977724, 3.31272409, 3.49850788, 3.21856403, 2.81659623,
        3.53931336],
       ...,
       [3.0943008 , 2.87959278, 2.96566776, 2.7831038 , 3.16889561,
        2.72503533],
       [3.55449927, 3.39419441, 3.85520167, 3.63479312, 3.53371769,
        3.46682633],
       [2.45129822, 2.33623625, 2.66637986, 2.50908584, 2.0604793 ,
        2.61086487]])

In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.713236,2.702874,2.773366,2.861172,2.525257,2.796407
1,3.0,2.0,3.0,3.5,3.0,3.0,3.172907,2.937720,3.000230,2.871164,2.985416,2.906556
2,4.0,4.0,3.0,4.0,4.0,4.0,3.019777,3.312724,3.498508,3.218564,2.816596,3.539313
3,3.0,3.0,3.5,3.0,3.5,3.5,3.093195,2.939164,3.314712,3.097877,3.067471,3.084654
4,3.5,3.5,3.5,3.5,3.0,3.5,3.465154,3.281809,3.264400,3.303183,3.093213,3.059540
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.183771,3.361566,3.628851,3.691744,3.824559,3.303886
779,2.5,2.5,3.0,3.0,2.5,2.5,2.922684,3.008662,3.122402,3.073830,3.177676,3.028604
780,2.0,3.0,3.0,3.0,3.0,2.5,3.094301,2.879593,2.965668,2.783104,3.168896,2.725035
781,4.0,3.5,4.0,3.5,3.5,4.0,3.554499,3.394194,3.855202,3.634793,3.533718,3.466826


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.713236,2.702874,2.773366,2.861172,2.525257,2.796407,0.082233,0.041158,7.472914e-02,0.741618,0.275895,0.634264
1,3.0,2.0,3.0,3.5,3.0,3.0,3.172907,2.937720,3.000230,2.871164,2.985416,2.906556,0.029897,0.879319,5.294000e-08,0.395435,0.000213,0.008732
2,4.0,4.0,3.0,4.0,4.0,4.0,3.019777,3.312724,3.498508,3.218564,2.816596,3.539313,0.960837,0.472348,2.485101e-01,0.610642,1.400444,0.212232
3,3.0,3.0,3.5,3.0,3.5,3.5,3.093195,2.939164,3.314712,3.097877,3.067471,3.084654,0.008685,0.003701,3.433146e-02,0.009580,0.187082,0.172512
4,3.5,3.5,3.5,3.5,3.0,3.5,3.465154,3.281809,3.264400,3.303183,3.093213,3.059540,0.001214,0.047607,5.550735e-02,0.038737,0.008689,0.194005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.183771,3.361566,3.628851,3.691744,3.824559,3.303886,1.401315,0.742296,3.954539e-01,0.478510,0.105339,1.700118
779,2.5,2.5,3.0,3.0,2.5,2.5,2.922684,3.008662,3.122402,3.073830,3.177676,3.028604,0.178662,0.258738,1.498231e-02,0.005451,0.459245,0.279423
780,2.0,3.0,3.0,3.0,3.0,2.5,3.094301,2.879593,2.965668,2.783104,3.168896,2.725035,1.197494,0.014498,1.178702e-03,0.047044,0.028526,0.050641
781,4.0,3.5,4.0,3.5,3.5,4.0,3.554499,3.394194,3.855202,3.634793,3.533718,3.466826,0.198471,0.011195,2.096656e-02,0.018169,0.001137,0.284274


In [ ]:
mse = results.mean();mse

se_cohesion       0.303440
se_syntax         0.304997
se_vocabulary     0.242882
se_phraseology    0.299454
se_grammar        0.361495
se_conventions    0.334471
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.550854
se_syntax         0.552265
se_vocabulary     0.492831
se_phraseology    0.547224
se_grammar        0.601244
se_conventions    0.578335
dtype: float64

In [ ]:
rmse.mean()

0.5537921813304042

**LGBM Multi Output Regressor para X_train_custom**

In [ ]:
params_lgb = {
    "n_estimators": 1000,
    "verbose": -1
}

In [ ]:
%%time
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train_custom, y_train)

CPU times: user 24.6 s, sys: 323 ms, total: 25 s
Wall time: 6.25 s


MultiOutputRegressor(estimator=LGBMRegressor(n_estimators=1000, verbose=-1))

In [ ]:
y_pred = model.predict(X_test_custom)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.6517223579194497
0.5218117857226926


In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.144665,2.342654,2.469640,1.910596,1.943460,1.991456
1,3.0,2.0,3.0,3.5,3.0,3.0,2.462441,2.445542,2.303691,2.213185,2.375107,2.188684
2,4.0,4.0,3.0,4.0,4.0,4.0,3.449226,2.811174,3.260435,3.012006,3.387095,3.299179
3,3.0,3.0,3.5,3.0,3.5,3.5,3.348455,3.199169,3.268753,3.443077,3.618974,3.477908
4,3.5,3.5,3.5,3.5,3.0,3.5,2.233064,2.633177,2.498885,2.343928,2.407352,2.038784
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.806977,2.843493,3.285335,2.642713,2.783188,2.143987
779,2.5,2.5,3.0,3.0,2.5,2.5,3.014735,3.012533,2.997487,3.084838,3.366772,2.924655
780,2.0,3.0,3.0,3.0,3.0,2.5,2.723049,3.041111,3.327487,3.071428,3.171507,3.544552
781,4.0,3.5,4.0,3.5,3.5,4.0,3.532585,3.494007,3.907405,3.895244,3.654625,3.694671


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.144665,2.342654,2.469640,1.910596,1.943460,1.991456,0.731598,0.024758,0.000922,0.007993,0.003197,0.000073
1,3.0,2.0,3.0,3.5,3.0,3.0,2.462441,2.445542,2.303691,2.213185,2.375107,2.188684,0.288970,0.198508,0.484846,1.655893,0.390491,0.658234
2,4.0,4.0,3.0,4.0,4.0,4.0,3.449226,2.811174,3.260435,3.012006,3.387095,3.299179,0.303352,1.413307,0.067826,0.976133,0.375653,0.491150
3,3.0,3.0,3.5,3.0,3.5,3.5,3.348455,3.199169,3.268753,3.443077,3.618974,3.477908,0.121421,0.039668,0.053475,0.196317,0.014155,0.000488
4,3.5,3.5,3.5,3.5,3.0,3.5,2.233064,2.633177,2.498885,2.343928,2.407352,2.038784,1.605126,0.751382,1.002231,1.336502,0.351232,2.135153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.806977,2.843493,3.285335,2.642713,2.783188,2.143987,0.651212,0.117987,0.081416,0.127654,0.513819,0.020732
779,2.5,2.5,3.0,3.0,2.5,2.5,3.014735,3.012533,2.997487,3.084838,3.366772,2.924655,0.264952,0.262690,0.000006,0.007197,0.751293,0.180332
780,2.0,3.0,3.0,3.0,3.0,2.5,2.723049,3.041111,3.327487,3.071428,3.171507,3.544552,0.522800,0.001690,0.107248,0.005102,0.029415,1.091090
781,4.0,3.5,4.0,3.5,3.5,4.0,3.532585,3.494007,3.907405,3.895244,3.654625,3.694671,0.218476,0.000036,0.008574,0.156218,0.023909,0.093226


In [ ]:
mse = results.mean();mse

se_cohesion       0.418684
se_syntax         0.403569
se_vocabulary     0.324779
se_phraseology    0.448978
se_grammar        0.493866
se_conventions    0.469633
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.647058
se_syntax         0.635271
se_vocabulary     0.569894
se_phraseology    0.670058
se_grammar        0.702756
se_conventions    0.685298
dtype: float64

In [ ]:
rmse.mean()

0.6517223579194497

**LGBM Cross Validated para X_train**

In [ ]:
params_lgb = {'subsample_freq': 0, 'reg_lambda': 61, 'reg_alpha': 21, 'num_leaves': 85, 'num_iterations': 110, 'n_estimators': 4900, 'min_data_in_leaf': 11, 'max_depth': 30, 'max_bin': 100, 'learning_rate': 0.1, 'importance_type': 'gain', 'feature_fraction': 0.2, 'boosting_type': 'gbdt', 'bagging_fraction': 0.3}

In [ ]:
%%time
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.808764,2.574076,2.730523,2.611417,2.440365,2.607504
1,3.0,2.0,3.0,3.5,3.0,3.0,3.097107,3.151163,3.079234,3.082309,3.040330,3.100287
2,4.0,4.0,3.0,4.0,4.0,4.0,3.234818,3.227814,3.329093,3.214551,3.059031,3.317234
3,3.0,3.0,3.5,3.0,3.5,3.5,3.212348,3.042361,3.407557,3.185627,3.102372,3.288398
4,3.5,3.5,3.5,3.5,3.0,3.5,3.228003,3.131025,3.358188,3.319554,3.202069,3.102000
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.241960,3.358034,3.485457,3.462869,3.627058,3.280356
779,2.5,2.5,3.0,3.0,2.5,2.5,2.919324,2.993501,3.174211,3.093263,3.173311,2.926033
780,2.0,3.0,3.0,3.0,3.0,2.5,2.988276,2.899052,3.071076,3.011590,2.819662,2.798988
781,4.0,3.5,4.0,3.5,3.5,4.0,3.418566,3.357025,3.686622,3.474851,3.218166,3.351108


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.808764,2.574076,2.730523,2.611417,2.440365,2.607504,0.036571,0.005487,0.053141,0.373831,0.193922,0.369061
1,3.0,2.0,3.0,3.5,3.0,3.0,3.097107,3.151163,3.079234,3.082309,3.040330,3.100287,0.009430,1.325176,0.006278,0.174466,0.001626,0.010057
2,4.0,4.0,3.0,4.0,4.0,4.0,3.234818,3.227814,3.329093,3.214551,3.059031,3.317234,0.585504,0.596272,0.108302,0.616930,0.885423,0.466169
3,3.0,3.0,3.5,3.0,3.5,3.5,3.212348,3.042361,3.407557,3.185627,3.102372,3.288398,0.045092,0.001794,0.008546,0.034457,0.158108,0.044775
4,3.5,3.5,3.5,3.5,3.0,3.5,3.228003,3.131025,3.358188,3.319554,3.202069,3.102000,0.073982,0.136142,0.020111,0.032561,0.040832,0.158404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.241960,3.358034,3.485457,3.462869,3.627058,3.280356,1.542464,0.736221,0.235668,0.214247,0.016144,1.639310
779,2.5,2.5,3.0,3.0,2.5,2.5,2.919324,2.993501,3.174211,3.093263,3.173311,2.926033,0.175833,0.243543,0.030349,0.008698,0.453348,0.181504
780,2.0,3.0,3.0,3.0,3.0,2.5,2.988276,2.899052,3.071076,3.011590,2.819662,2.798988,0.976689,0.010191,0.005052,0.000134,0.032522,0.089394
781,4.0,3.5,4.0,3.5,3.5,4.0,3.418566,3.357025,3.686622,3.474851,3.218166,3.351108,0.338066,0.020442,0.098206,0.000632,0.079431,0.421061


In [ ]:
mse = results.mean();mse

se_cohesion       0.298038
se_syntax         0.298765
se_vocabulary     0.234480
se_phraseology    0.290553
se_grammar        0.341597
se_conventions    0.331695
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.545928
se_syntax         0.546594
se_vocabulary     0.484231
se_phraseology    0.539029
se_grammar        0.584463
se_conventions    0.575930
dtype: float64

In [ ]:
rmse.mean()

0.546029288853226

**LGBM Cross Validated para X_train_custome**

In [ ]:
params_lgb = {'subsample_freq': 0, 'reg_lambda': 61, 'reg_alpha': 21, 'num_leaves': 85, 'num_iterations': 110, 'n_estimators': 4900, 'min_data_in_leaf': 11, 'max_depth': 30, 'max_bin': 100, 'learning_rate': 0.1, 'importance_type': 'gain', 'feature_fraction': 0.2, 'boosting_type': 'gbdt', 'bagging_fraction': 0.3}

In [ ]:
%%time
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train_custom, y_train)

In [ ]:
y_pred = model.predict(X_test_custom)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.826064,2.559371,2.849160,2.610026,2.600051,2.475609
1,3.0,2.0,3.0,3.5,3.0,3.0,2.689889,2.640157,2.800574,2.671045,2.659738,2.611200
2,4.0,4.0,3.0,4.0,4.0,4.0,3.365987,3.245008,3.404841,3.262921,3.219115,3.239994
3,3.0,3.0,3.5,3.0,3.5,3.5,3.223679,3.099209,3.272141,3.154422,3.091760,3.086370
4,3.5,3.5,3.5,3.5,3.0,3.5,2.763573,2.632366,2.970422,2.697157,2.678730,2.459624
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.850075,2.826068,3.043942,2.789255,2.739977,2.516406
779,2.5,2.5,3.0,3.0,2.5,2.5,2.790739,2.819050,2.938674,2.961967,3.030479,2.987198
780,2.0,3.0,3.0,3.0,3.0,2.5,3.174656,3.095124,3.349865,3.078814,2.964512,3.352411
781,4.0,3.5,4.0,3.5,3.5,4.0,3.452288,3.372118,3.692179,3.467250,3.212293,3.517624


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.826064,2.559371,2.849160,2.610026,2.600051,2.475609,0.030254,0.003525,0.121913,0.372131,0.360061,0.226204
1,3.0,2.0,3.0,3.5,3.0,3.0,2.689889,2.640157,2.800574,2.671045,2.659738,2.611200,0.096169,0.409801,0.039771,0.687166,0.115778,0.151165
2,4.0,4.0,3.0,4.0,4.0,4.0,3.365987,3.245008,3.404841,3.262921,3.219115,3.239994,0.401973,0.570013,0.163897,0.543286,0.609782,0.577609
3,3.0,3.0,3.5,3.0,3.5,3.5,3.223679,3.099209,3.272141,3.154422,3.091760,3.086370,0.050032,0.009842,0.051920,0.023846,0.166660,0.171090
4,3.5,3.5,3.5,3.5,3.0,3.5,2.763573,2.632366,2.970422,2.697157,2.678730,2.459624,0.542324,0.752788,0.280453,0.644557,0.103215,1.082382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.850075,2.826068,3.043942,2.789255,2.739977,2.516406,0.722627,0.106320,0.001931,0.044413,0.577635,0.266676
779,2.5,2.5,3.0,3.0,2.5,2.5,2.790739,2.819050,2.938674,2.961967,3.030479,2.987198,0.084529,0.101793,0.003761,0.001446,0.281408,0.237362
780,2.0,3.0,3.0,3.0,3.0,2.5,3.174656,3.095124,3.349865,3.078814,2.964512,3.352411,1.379816,0.009049,0.122405,0.006212,0.001259,0.726605
781,4.0,3.5,4.0,3.5,3.5,4.0,3.452288,3.372118,3.692179,3.467250,3.212293,3.517624,0.299989,0.016354,0.094754,0.001073,0.082775,0.232687


In [ ]:
mse = results.mean();mse

se_cohesion       0.321012
se_syntax         0.309616
se_vocabulary     0.253780
se_phraseology    0.340302
se_grammar        0.416135
se_conventions    0.339079
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.566579
se_syntax         0.556432
se_vocabulary     0.503766
se_phraseology    0.583354
se_grammar        0.645085
se_conventions    0.582305
dtype: float64

In [ ]:
rmse.mean()

0.5729202809714039

## 2. Regresión Lineal

In [ ]:
from sklearn.linear_model import LinearRegression

**Con X_train**

In [ ]:
params_reg = {
    "n_estimators": 1000,
    "verbose": -1
}

In [ ]:
%%time
model = MultiOutputRegressor(LinearRegression())
model.fit(X_train, y_train)

CPU times: user 8min 31s, sys: 3min 57s, total: 12min 28s
Wall time: 1min 56s


MultiOutputRegressor(estimator=LinearRegression())

In [ ]:
y_pred = model.predict(X_test)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.725830013486247
0.579226035218831


In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,3.499931,3.303012,3.400357,3.484552,3.217331,3.107439
1,3.0,2.0,3.0,3.5,3.0,3.0,3.107949,3.357546,3.058578,2.399447,3.956310,3.321345
2,4.0,4.0,3.0,4.0,4.0,4.0,3.696971,3.644065,3.720008,3.571838,2.740218,3.385166
3,3.0,3.0,3.5,3.0,3.5,3.5,2.216299,3.242095,2.804115,2.374760,2.550498,1.948568
4,3.5,3.5,3.5,3.5,3.0,3.5,2.402935,2.871715,3.094099,2.763581,2.656512,3.329135
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.289074,2.895573,3.311062,2.631673,2.637095,1.804333
779,2.5,2.5,3.0,3.0,2.5,2.5,3.360661,3.604091,3.152435,3.663565,2.778330,4.014325
780,2.0,3.0,3.0,3.0,3.0,2.5,2.611258,2.628411,2.685446,2.705634,3.587196,2.715121
781,4.0,3.5,4.0,3.5,3.5,4.0,4.050897,3.631106,4.416546,4.406541,3.485710,3.903174


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,3.499931,3.303012,3.400357,3.484552,3.217331,3.107439,0.249931,0.644829,0.810643,2.203895,1.481896,1.226420
1,3.0,2.0,3.0,3.5,3.0,3.0,3.107949,3.357546,3.058578,2.399447,3.956310,3.321345,0.011653,1.842932,0.003431,1.211218,0.914529,0.103262
2,4.0,4.0,3.0,4.0,4.0,4.0,3.696971,3.644065,3.720008,3.571838,2.740218,3.385166,0.091827,0.126689,0.518411,0.183323,1.587051,0.378021
3,3.0,3.0,3.5,3.0,3.5,3.5,2.216299,3.242095,2.804115,2.374760,2.550498,1.948568,0.614188,0.058610,0.484256,0.390925,0.901554,2.406942
4,3.5,3.5,3.5,3.5,3.0,3.5,2.402935,2.871715,3.094099,2.763581,2.656512,3.329135,1.203552,0.394741,0.164756,0.542313,0.117984,0.029195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.289074,2.895573,3.311062,2.631673,2.637095,1.804333,1.661712,0.156478,0.096760,0.135665,0.744605,0.038286
779,2.5,2.5,3.0,3.0,2.5,2.5,3.360661,3.604091,3.152435,3.663565,2.778330,4.014325,0.740738,1.219017,0.023236,0.440318,0.077468,2.293179
780,2.0,3.0,3.0,3.0,3.0,2.5,2.611258,2.628411,2.685446,2.705634,3.587196,2.715121,0.373636,0.138078,0.098944,0.086651,0.344799,0.046277
781,4.0,3.5,4.0,3.5,3.5,4.0,4.050897,3.631106,4.416546,4.406541,3.485710,3.903174,0.002591,0.017189,0.173510,0.821816,0.000204,0.009375


In [ ]:
mse = results.mean();mse

se_cohesion       0.567317
se_syntax         0.503309
se_vocabulary     0.394060
se_phraseology    0.500830
se_grammar        0.651391
se_conventions    0.562214
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.753205
se_syntax         0.709442
se_vocabulary     0.627742
se_phraseology    0.707693
se_grammar        0.807088
se_conventions    0.749809
dtype: float64

In [ ]:
rmse.mean()

0.725830013486247

**Con X_train_custom**

In [ ]:
params_reg = {
    "n_estimators": 1000,
    "verbose": -1
}

In [ ]:
%%time
model = MultiOutputRegressor(LinearRegression())
model.fit(X_train_custom, y_train)

CPU times: user 63.5 ms, sys: 171 ms, total: 235 ms
Wall time: 35.8 ms


MultiOutputRegressor(estimator=LinearRegression())

In [ ]:
y_pred = model.predict(X_test_custom)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.6993617560418329
0.5547843250573176


In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.628775,2.626038,2.162255,2.043799,1.614254,1.225849
1,3.0,2.0,3.0,3.5,3.0,3.0,3.007920,3.025414,2.694529,2.969092,2.670459,2.463990
2,4.0,4.0,3.0,4.0,4.0,4.0,4.438674,4.349455,3.833715,4.000374,3.549370,3.471880
3,3.0,3.0,3.5,3.0,3.5,3.5,3.292676,3.396886,3.055798,2.967635,2.869549,2.777281
4,3.5,3.5,3.5,3.5,3.0,3.5,3.092457,3.153493,2.825421,2.893925,2.621232,1.986130
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.486876,3.482403,3.053989,3.250845,2.923793,2.109293
779,2.5,2.5,3.0,3.0,2.5,2.5,3.462749,3.549367,3.180833,3.306320,3.238870,2.959493
780,2.0,3.0,3.0,3.0,3.0,2.5,2.941944,3.086295,2.474970,2.035725,2.004409,2.569026
781,4.0,3.5,4.0,3.5,3.5,4.0,4.609639,4.567072,4.090644,4.356445,3.897151,3.688733


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.628775,2.626038,2.162255,2.043799,1.614254,1.225849,0.137808,0.015886,0.114072,1.918336e-03,0.148800,0.599310
1,3.0,2.0,3.0,3.5,3.0,3.0,3.007920,3.025414,2.694529,2.969092,2.670459,2.463990,0.000063,1.051474,0.093312,2.818635e-01,0.108597,0.287306
2,4.0,4.0,3.0,4.0,4.0,4.0,4.438674,4.349455,3.833715,4.000374,3.549370,3.471880,0.192434,0.122119,0.695080,1.399258e-07,0.203067,0.278910
3,3.0,3.0,3.5,3.0,3.5,3.5,3.292676,3.396886,3.055798,2.967635,2.869549,2.777281,0.085659,0.157518,0.197316,1.047505e-03,0.397468,0.522323
4,3.5,3.5,3.5,3.5,3.0,3.5,3.092457,3.153493,2.825421,2.893925,2.621232,1.986130,0.166091,0.120067,0.455057,3.673273e-01,0.143465,2.291803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.486876,3.482403,3.053989,3.250845,2.923793,2.109293,2.210800,0.965116,0.002915,6.292332e-02,0.332014,0.011945
779,2.5,2.5,3.0,3.0,2.5,2.5,3.462749,3.549367,3.180833,3.306320,3.238870,2.959493,0.926885,1.101172,0.032701,9.383202e-02,0.545930,0.211134
780,2.0,3.0,3.0,3.0,3.0,2.5,2.941944,3.086295,2.474970,2.035725,2.004409,2.569026,0.887258,0.007447,0.275657,9.298259e-01,0.991201,0.004765
781,4.0,3.5,4.0,3.5,3.5,4.0,4.609639,4.567072,4.090644,4.356445,3.897151,3.688733,0.371659,1.138643,0.008216,7.334984e-01,0.157729,0.096887


In [ ]:
mse = results.mean();mse

se_cohesion       0.583377
se_syntax         0.626407
se_vocabulary     0.304148
se_phraseology    0.413079
se_grammar        0.448880
se_conventions    0.603305
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.763791
se_syntax         0.791459
se_vocabulary     0.551496
se_phraseology    0.642712
se_grammar        0.669985
se_conventions    0.776727
dtype: float64

In [ ]:
rmse.mean()

0.6993617560418329

## 3. XGBoost

In [ ]:
import xgboost as xgb

**XGBoost para X_train**

In [ ]:
params_xgb = {'n_estimators': 1000, 'random_state':0}

In [ ]:
%%time
model = MultiOutputRegressor(xgb.XGBRegressor(
        **params_xgb,
        objective='reg:squarederror'))
model.fit(X_train, y_train)

CPU times: user 1h 42min 5s, sys: 6.69 s, total: 1h 42min 12s
Wall time: 14min 32s


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=0, ...))

In [ ]:
y_pred = model.predict(X_test)
print(RMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.5829136795459533
0.4654132361968257


In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.743224,2.731650,2.982111,2.714278,2.397123,2.985538
1,3.0,2.0,3.0,3.5,3.0,3.0,2.828160,3.223107,3.114131,3.052536,3.187719,2.776478
2,4.0,4.0,3.0,4.0,4.0,4.0,2.989537,3.264683,3.255399,3.295654,2.871751,3.393271
3,3.0,3.0,3.5,3.0,3.5,3.5,3.083305,2.777524,3.093509,3.146132,3.160378,3.114782
4,3.5,3.5,3.5,3.5,3.0,3.5,3.164616,3.602264,3.353054,2.987478,2.857738,2.966591
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.114060,3.393386,3.419405,3.332921,3.855688,3.416206
779,2.5,2.5,3.0,3.0,2.5,2.5,3.302265,3.064982,2.966523,3.264858,3.145991,3.127845
780,2.0,3.0,3.0,3.0,3.0,2.5,3.402995,2.813798,2.825599,3.081828,3.206443,2.884912
781,4.0,3.5,4.0,3.5,3.5,4.0,3.354503,3.106086,3.533335,3.769479,2.779988,3.712328


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.743224,2.731650,2.982111,2.714278,2.397123,2.985538,0.065934,0.053662,0.232431,0.510193,0.157707,0.971286
1,3.0,2.0,3.0,3.5,3.0,3.0,2.828160,3.223107,3.114131,3.052536,3.187719,2.776478,0.029529,1.495990,0.013026,0.200224,0.035238,0.049962
2,4.0,4.0,3.0,4.0,4.0,4.0,2.989537,3.264683,3.255399,3.295654,2.871751,3.393271,1.021035,0.540692,0.065228,0.496103,1.272945,0.368120
3,3.0,3.0,3.5,3.0,3.5,3.5,3.083305,2.777524,3.093509,3.146132,3.160378,3.114782,0.006940,0.049496,0.165235,0.021355,0.115343,0.148393
4,3.5,3.5,3.5,3.5,3.0,3.5,3.164616,3.602264,3.353054,2.987478,2.857738,2.966591,0.112483,0.010458,0.021593,0.262678,0.020238,0.284525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.114060,3.393386,3.419405,3.332921,3.855688,3.416206,1.241131,0.798139,0.175900,0.110837,0.126514,2.005640
779,2.5,2.5,3.0,3.0,2.5,2.5,3.302265,3.064982,2.966523,3.264858,3.145991,3.127845,0.643629,0.319205,0.001121,0.070150,0.417305,0.394189
780,2.0,3.0,3.0,3.0,3.0,2.5,3.402995,2.813798,2.825599,3.081828,3.206443,2.884912,1.968396,0.034671,0.030416,0.006696,0.042619,0.148157
781,4.0,3.5,4.0,3.5,3.5,4.0,3.354503,3.106086,3.533335,3.769479,2.779988,3.712328,0.416666,0.155168,0.217776,0.072619,0.518418,0.082755


In [ ]:
mse = results.mean();mse

se_cohesion       0.337070
se_syntax         0.348157
se_vocabulary     0.256047
se_phraseology    0.318577
se_grammar        0.396127
se_conventions    0.382752
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.580577
se_syntax         0.590048
se_vocabulary     0.506011
se_phraseology    0.564426
se_grammar        0.629386
se_conventions    0.618669
dtype: float64

In [ ]:
rmse.mean()

0.5815197701950573

**XGBoost para X_train_custome**

In [ ]:
params_xgb = {'n_estimators': 1000, 'random_state':0}

In [ ]:
%%time
model = MultiOutputRegressor(xgb.XGBRegressor(
        **params_xgb,
        objective='reg:squarederror'))
model.fit(X_train_custom, y_train)

CPU times: user 1min 8s, sys: 99.6 ms, total: 1min 8s
Wall time: 9.06 s


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=0, ...))

In [ ]:
y_pred = model.predict(X_test_custom)
print(RMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.677429717374932
0.5380674134979556


In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.317039,2.535920,1.888835,2.177927,2.099444,2.134776
1,3.0,2.0,3.0,3.5,3.0,3.0,2.546365,2.645905,2.222825,2.429929,2.035377,2.174752
2,4.0,4.0,3.0,4.0,4.0,4.0,3.794694,2.831709,3.141286,3.286137,3.198235,3.429970
3,3.0,3.0,3.5,3.0,3.5,3.5,3.349554,3.338071,3.148988,3.437922,3.472612,3.167054
4,3.5,3.5,3.5,3.5,3.0,3.5,2.138792,2.387981,2.879414,2.436935,3.019927,1.669137
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.876550,2.786259,3.754968,2.366302,2.462768,2.357132
779,2.5,2.5,3.0,3.0,2.5,2.5,2.876534,3.033310,3.331741,2.936902,3.634569,3.041547
780,2.0,3.0,3.0,3.0,3.0,2.5,2.746247,3.125446,2.938615,3.130644,3.048948,3.626090
781,4.0,3.5,4.0,3.5,3.5,4.0,3.114086,3.339130,3.640980,4.011469,3.186199,3.127144


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.317039,2.535920,1.888835,2.177927,2.099444,2.134776,0.466435,0.001290,0.373523,0.031658,0.009889,0.018165
1,3.0,2.0,3.0,3.5,3.0,3.0,2.546365,2.645905,2.222825,2.429929,2.035377,2.174752,0.205785,0.417193,0.604001,1.145051,0.930497,0.681035
2,4.0,4.0,3.0,4.0,4.0,4.0,3.794694,2.831709,3.141286,3.286137,3.198235,3.429970,0.042150,1.364904,0.019962,0.509601,0.642827,0.324934
3,3.0,3.0,3.5,3.0,3.5,3.5,3.349554,3.338071,3.148988,3.437922,3.472612,3.167054,0.122188,0.114292,0.123209,0.191775,0.000750,0.110853
4,3.5,3.5,3.5,3.5,3.0,3.5,2.138792,2.387981,2.879414,2.436935,3.019927,1.669137,1.852887,1.236585,0.385127,1.130107,0.000397,3.352061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.876550,2.786259,3.754968,2.366302,2.462768,2.357132,0.768341,0.081944,0.569977,0.401573,1.075850,0.127543
779,2.5,2.5,3.0,3.0,2.5,2.5,2.876534,3.033310,3.331741,2.936902,3.634569,3.041547,0.141778,0.284419,0.110052,0.003981,1.287247,0.293273
780,2.0,3.0,3.0,3.0,3.0,2.5,2.746247,3.125446,2.938615,3.130644,3.048948,3.626090,0.556884,0.015737,0.003768,0.017068,0.002396,1.268078
781,4.0,3.5,4.0,3.5,3.5,4.0,3.114086,3.339130,3.640980,4.011469,3.186199,3.127144,0.784844,0.025879,0.128895,0.261600,0.098471,0.761878


In [ ]:
mse = results.mean();mse

se_cohesion       0.453241
se_syntax         0.409277
se_vocabulary     0.359731
se_phraseology    0.476806
se_grammar        0.544697
se_conventions    0.509713
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.673232
se_syntax         0.639748
se_vocabulary     0.599776
se_phraseology    0.690512
se_grammar        0.738036
se_conventions    0.713942
dtype: float64

In [ ]:
rmse.mean()

0.6758741981642138

**XGBoost Cross Validated para X_train**

In [ ]:
params_xgb_cv =  {'subsample': 0.8, 'reg_alpha': 0, 'n_estimators': 740, 'min_child_weight': 12, 'max_depth': 6, 'lambda': 5, 'gamma': 0.3, 'eval_metric': 'rmse', 'eta': 0.03, 'colsample_bytree': 0.85, 'booster': 'gbtree'}

In [ ]:
%%time
model = MultiOutputRegressor(xgb.XGBRegressor(
        **params_xgb,
        objective='reg:squarederror'))
model.fit(X_train, y_train)

CPU times: user 1h 41min 9s, sys: 4.41 s, total: 1h 41min 13s
Wall time: 14min 19s


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=0, ...))

In [ ]:
y_pred = model.predict(X_test)
print(RMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.5829136795459533
0.4654132361968257


In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.743224,2.731650,2.982111,2.714278,2.397123,2.985538
1,3.0,2.0,3.0,3.5,3.0,3.0,2.828160,3.223107,3.114131,3.052536,3.187719,2.776478
2,4.0,4.0,3.0,4.0,4.0,4.0,2.989537,3.264683,3.255399,3.295654,2.871751,3.393271
3,3.0,3.0,3.5,3.0,3.5,3.5,3.083305,2.777524,3.093509,3.146132,3.160378,3.114782
4,3.5,3.5,3.5,3.5,3.0,3.5,3.164616,3.602264,3.353054,2.987478,2.857738,2.966591
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.114060,3.393386,3.419405,3.332921,3.855688,3.416206
779,2.5,2.5,3.0,3.0,2.5,2.5,3.302265,3.064982,2.966523,3.264858,3.145991,3.127845
780,2.0,3.0,3.0,3.0,3.0,2.5,3.402995,2.813798,2.825599,3.081828,3.206443,2.884912
781,4.0,3.5,4.0,3.5,3.5,4.0,3.354503,3.106086,3.533335,3.769479,2.779988,3.712328


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.743224,2.731650,2.982111,2.714278,2.397123,2.985538,0.065934,0.053662,0.232431,0.510193,0.157707,0.971286
1,3.0,2.0,3.0,3.5,3.0,3.0,2.828160,3.223107,3.114131,3.052536,3.187719,2.776478,0.029529,1.495990,0.013026,0.200224,0.035238,0.049962
2,4.0,4.0,3.0,4.0,4.0,4.0,2.989537,3.264683,3.255399,3.295654,2.871751,3.393271,1.021035,0.540692,0.065228,0.496103,1.272945,0.368120
3,3.0,3.0,3.5,3.0,3.5,3.5,3.083305,2.777524,3.093509,3.146132,3.160378,3.114782,0.006940,0.049496,0.165235,0.021355,0.115343,0.148393
4,3.5,3.5,3.5,3.5,3.0,3.5,3.164616,3.602264,3.353054,2.987478,2.857738,2.966591,0.112483,0.010458,0.021593,0.262678,0.020238,0.284525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.114060,3.393386,3.419405,3.332921,3.855688,3.416206,1.241131,0.798139,0.175900,0.110837,0.126514,2.005640
779,2.5,2.5,3.0,3.0,2.5,2.5,3.302265,3.064982,2.966523,3.264858,3.145991,3.127845,0.643629,0.319205,0.001121,0.070150,0.417305,0.394189
780,2.0,3.0,3.0,3.0,3.0,2.5,3.402995,2.813798,2.825599,3.081828,3.206443,2.884912,1.968396,0.034671,0.030416,0.006696,0.042619,0.148157
781,4.0,3.5,4.0,3.5,3.5,4.0,3.354503,3.106086,3.533335,3.769479,2.779988,3.712328,0.416666,0.155168,0.217776,0.072619,0.518418,0.082755


In [ ]:
mse = results.mean();mse

se_cohesion       0.337070
se_syntax         0.348157
se_vocabulary     0.256047
se_phraseology    0.318577
se_grammar        0.396127
se_conventions    0.382752
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.580577
se_syntax         0.590048
se_vocabulary     0.506011
se_phraseology    0.564426
se_grammar        0.629386
se_conventions    0.618669
dtype: float64

In [ ]:
rmse.mean()

0.5815197701950573

**XGBoost Cross Validated para X_train_custome**

In [ ]:
params_xgb_cv =  {'subsample': 0.8, 'reg_alpha': 0, 'n_estimators': 740, 'min_child_weight': 12, 'max_depth': 6, 'lambda': 5, 'gamma': 0.3, 'eval_metric': 'rmse', 'eta': 0.03, 'colsample_bytree': 0.85, 'booster': 'gbtree'}

In [ ]:
%%time
model = MultiOutputRegressor(xgb.XGBRegressor(
        **params_xgb,
        objective='reg:squarederror'))
model.fit(X_train_custom, y_train)

CPU times: user 2min 11s, sys: 125 ms, total: 2min 11s
Wall time: 19 s


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=0, ...))

In [ ]:
y_pred = model.predict(X_test_custom)
print(RMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.677429717374932
0.5380674134979556


In [ ]:
preds = pd.DataFrame(y_pred, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= y_test.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.317039,2.535920,1.888835,2.177927,2.099444,2.134776
1,3.0,2.0,3.0,3.5,3.0,3.0,2.546365,2.645905,2.222825,2.429929,2.035377,2.174752
2,4.0,4.0,3.0,4.0,4.0,4.0,3.794694,2.831709,3.141286,3.286137,3.198235,3.429970
3,3.0,3.0,3.5,3.0,3.5,3.5,3.349554,3.338071,3.148988,3.437922,3.472612,3.167054
4,3.5,3.5,3.5,3.5,3.0,3.5,2.138792,2.387981,2.879414,2.436935,3.019927,1.669137
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.876550,2.786259,3.754968,2.366302,2.462768,2.357132
779,2.5,2.5,3.0,3.0,2.5,2.5,2.876534,3.033310,3.331741,2.936902,3.634569,3.041547
780,2.0,3.0,3.0,3.0,3.0,2.5,2.746247,3.125446,2.938615,3.130644,3.048948,3.626090
781,4.0,3.5,4.0,3.5,3.5,4.0,3.114086,3.339130,3.640980,4.011469,3.186199,3.127144


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.317039,2.535920,1.888835,2.177927,2.099444,2.134776,0.466435,0.001290,0.373523,0.031658,0.009889,0.018165
1,3.0,2.0,3.0,3.5,3.0,3.0,2.546365,2.645905,2.222825,2.429929,2.035377,2.174752,0.205785,0.417193,0.604001,1.145051,0.930497,0.681035
2,4.0,4.0,3.0,4.0,4.0,4.0,3.794694,2.831709,3.141286,3.286137,3.198235,3.429970,0.042150,1.364904,0.019962,0.509601,0.642827,0.324934
3,3.0,3.0,3.5,3.0,3.5,3.5,3.349554,3.338071,3.148988,3.437922,3.472612,3.167054,0.122188,0.114292,0.123209,0.191775,0.000750,0.110853
4,3.5,3.5,3.5,3.5,3.0,3.5,2.138792,2.387981,2.879414,2.436935,3.019927,1.669137,1.852887,1.236585,0.385127,1.130107,0.000397,3.352061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.876550,2.786259,3.754968,2.366302,2.462768,2.357132,0.768341,0.081944,0.569977,0.401573,1.075850,0.127543
779,2.5,2.5,3.0,3.0,2.5,2.5,2.876534,3.033310,3.331741,2.936902,3.634569,3.041547,0.141778,0.284419,0.110052,0.003981,1.287247,0.293273
780,2.0,3.0,3.0,3.0,3.0,2.5,2.746247,3.125446,2.938615,3.130644,3.048948,3.626090,0.556884,0.015737,0.003768,0.017068,0.002396,1.268078
781,4.0,3.5,4.0,3.5,3.5,4.0,3.114086,3.339130,3.640980,4.011469,3.186199,3.127144,0.784844,0.025879,0.128895,0.261600,0.098471,0.761878


In [ ]:
mse = results.mean();mse

se_cohesion       0.453241
se_syntax         0.409277
se_vocabulary     0.359731
se_phraseology    0.476806
se_grammar        0.544697
se_conventions    0.509713
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.673232
se_syntax         0.639748
se_vocabulary     0.599776
se_phraseology    0.690512
se_grammar        0.738036
se_conventions    0.713942
dtype: float64

In [ ]:
rmse.mean()

0.6758741981642138